# Design of Experiments and Optimization: Pellet mill
**Design of Experiments (DoE)** is a key tool in diagnostic analytics which, together with **response surface method (RSM)**, extends to have a predictive ability.
It helps understanding the relationship between the explanatory variables (independent variables) and the response variable (dependent variable).
The response surface itself is also used in deriving an optimal settings in a process.

## What is in this notebook?
We provide a demo of how one steps from the diagnostic DoE, to the predictive RSM, and finally to do optimization.
The example is extremely simplified in order to point out crucial ideas visually.

## Getting response surface

In [1]:
import numpy as np
import pandas as pd
from pyscipopt import Model, quicksum

In [2]:
# Import the DoE table
datafile = 'btgdat.xlsx'
df = pd.read_excel(datafile, sheet_name='Factorial CRD 3 factor')
print(df)

    order  A  B  C    PDI treat
0       8 -1 -1 -1  89.55    LL
1      13  1 -1 -1  91.23    HL
2      11 -1  1 -1  91.02    LH
3      10  1  1 -1  95.46    HH
4       2 -1 -1  1  91.58    LL
5       1  1 -1  1  94.78    HL
6      12 -1  1  1  93.89    LH
7       6  1  1  1  98.68    HH
8      14 -1 -1 -1  89.07    LL
9       4  1 -1 -1  91.57    HL
10     15 -1  1 -1  90.47    LH
11      3  1  1 -1  94.47    HH
12     16 -1 -1  1  91.82    LL
13      7  1 -1  1  93.94    HL
14      9 -1  1  1  92.24    LH
15      5  1  1  1  98.02    HH


In [3]:
x = df[['A', 'B', 'C']].to_numpy().T
y = df['PDI'].to_numpy().T

m,n = np.shape(x) # m = number of factors, n = number of runs

In [4]:
# Construct the matrix X by its transpose.
## First, stack ones over the linear terms.
X = np.vstack((np.ones(n),x))
## Second, stack the above matrix above the quadratic terms.
for i in range(m):
    for j in range(i,m):
        X = np.vstack((X,x[i,:]*x[j,:]))
X = X.T

In [5]:
# Do quadratic regression 
u = np.linalg.lstsq(X.T@X,X.T@y)[0]
c = u[0]
b = u[1:1+m].reshape(-1,1)
A = np.zeros((m,m))
A[np.triu_indices(3)] = u[1+m:]
A = .5*(A + A.T)
q = lambda x: x.T@A@x + b.T@x + c

In [6]:
for i in range(n):
    print(f"Run#{i}:\t x={x[:,i]}\t  y={y[i]}\t  P={q(x[:,i])[0]}")

Run#0:	 x=[-1 -1 -1]	  y=89.55	  P=89.37187499999999
Run#1:	 x=[ 1 -1 -1]	  y=91.23	  P=91.33812499999998
Run#2:	 x=[-1  1 -1]	  y=91.02	  P=90.68312499999999
Run#3:	 x=[ 1  1 -1]	  y=95.46	  P=95.02687499999999
Run#4:	 x=[-1 -1  1]	  y=91.58	  P=91.638125
Run#5:	 x=[ 1 -1  1]	  y=94.78	  P=94.421875
Run#6:	 x=[-1  1  1]	  y=93.89	  P=93.12687499999997
Run#7:	 x=[1 1 1]	  y=98.68	  P=98.28812499999998
Run#8:	 x=[-1 -1 -1]	  y=89.07	  P=89.37187499999999
Run#9:	 x=[ 1 -1 -1]	  y=91.57	  P=91.33812499999998
Run#10:	 x=[-1  1 -1]	  y=90.47	  P=90.68312499999999
Run#11:	 x=[ 1  1 -1]	  y=94.47	  P=95.02687499999999
Run#12:	 x=[-1 -1  1]	  y=91.82	  P=91.638125
Run#13:	 x=[ 1 -1  1]	  y=93.94	  P=94.421875
Run#14:	 x=[-1  1  1]	  y=92.24	  P=93.12687499999997
Run#15:	 x=[1 1 1]	  y=98.02	  P=98.28812499999998


----

## Optimization: Maximizing PDI

In [7]:
maxpdi = Model()
xx = [maxpdi.addVar(vtype='C', lb=-1, ub=1, name=f"x{i}") for i in range(m)]
obj = maxpdi.addVar(vtype='C', name="obj")
maxpdi.setObjective(obj, sense='maximize')
maxpdi.addCons( obj <= 
                        quicksum(A[i][j]*xx[i]*xx[j] for i in range(m) for j in range(m))
                        + quicksum(b[i][0]*xx[i] for i in range(m)) + c )
maxpdi.hideOutput()
maxpdi.optimize()
SOL = maxpdi.getBestSol()
x_max = [SOL[xx[i]] for i in range(m)]
OBJ = SOL[obj]
print(f"The maximum PDI={OBJ} is achieved at x1={x_max[0]} x2={x_max[1]} x3={x_max[2]}.")

The maximum PDI=98.288125001 is achieved at x1=1.0 x2=1.0 x3=1.0.


----

## Optimization: Tracking a specific PDI value

Instead of the PDI value as high as possible, we may want a specific PDI value $p^{\ast}$ that we prefer our pellets to have.

Since we want to be as close to $p^{\ast}$ as possible, the **square distance** between $q(x)$ and $p^{\ast}$ is minimized:
$$ 
\begin{array}{cl}
\min \quad &J(x) := (q(x) - p^{\ast})^{2} \\
\text{s.t.}\quad & x \in [-1,1]^{m}.
\end{array}
$$

This is no longer a quadratic program, but is still a **convex optimization problem** (check its eigenvalues).
With the structure, we aim to use the projected gradient descent method to solve this problem.
Recalling that $q(x) = x^{\top}Ax + b^{\top} + c$, we have $\nabla q(x) = 2Ax + b$ and then
$$
\nabla J(x) = 2(q(x) - p^{\ast}) \nabla q(x) = 2(q(x) - p^{\ast})(2Ax + b).
$$
The **projected gradient descent method (PGDM)** with diminishing rate is described as follows.

- **Initialization** Pick an initial point $x$ and an initial learning rate $\alpha > 0$.
- **Repeat**
   * Compute: $q(x^{0})$
   * Update learning rate: $\alpha \leftarrow \alpha/2$
   * Gradient step: $x \leftarrow x - 2\alpha(q(x) - p^{\ast})(2Ax + b)$
   * Projection step:
       * For $i = 1,\dots,m$;
           - If $x_{i} \leq -1$, $x_{i} \leftarrow -1$;
           - If $x_{i} \geq 1$, $x_{i} \leftarrow 1$;
- **Until** A stopping criterion is satisfied.
- **Return** $x$

In [8]:
# Check the eigenvalues of A for convexity check.
np.linalg.eig(A)[0] >= 0

array([ True,  True,  True])

In [9]:
p_star = 94.0
J = lambda x: (q(x) - p_star)**2

XX = np.array([1,1,1.]).reshape((-1,1))
alpha = 0.001

for k in range(20):
    qq = q(XX)
    alpha /= 2
    XX -= 2*alpha*(qq - p_star)*(2*A@XX + b)
    # print(f"Step{k}\t x = {XX}")
    for i in range(m):
        if XX[i] <= -1:
            XX[i] = -1
        elif XX[i] >= 1:
            XX[i] = 1
    # print(f"After proj, x = {XX}")
print(f"x = {XX.reshape((-1,))} q(x) = {q(XX)}")

x = [0.96816063 0.96858384 0.96878871] q(x) = [[93.77224431]]


### The failure of OFF-THE-SHELF solvers
Eventhough PySCIPOpt is advertised to solve nonlinear problems, it has its limitation outside the quadratic realm.

The following shows that PySCIPOpt fails to track the value of $p^{\ast}$.

In [10]:
m2 = Model()
t = m2.addVar(vtype='C', name='t')
xx = [m2.addVar(vtype='C', lb=-1, ub=1, name=f'x{i}') for i in range(m)]
Obj = m2.addVar(vtype='C', name="obj")
m2.setObjective(obj, sense='minimize')
m2.addCons( (t - p_star)**2 <= Obj )
m2.addCons( t == 
                  quicksum(A[i][j]*xx[i]*xx[j] for i in range(m) for j in range(m))
                  + quicksum(b[i][0]*xx[i] for i in range(m)) + c )
m2.hideOutput()
m2.optimize()

In [11]:
SOL = m2.getBestSol()
print(SOL)

{'t': 98.288125001, 'x0': 1.0, 'x1': 1.0, 'x2': 1.0, 'obj': 100000.0}


Note that the solution from the PySCIPOpt does not make sense, as the value of `Obj` should at most be $(t-p^{\ast})^{2}$.

----